In [1]:
%%init_spark
launcher.master="yarn"
launcher.num_executors=6
launcher.executor_cores=2
launcher.executor_memory="1600m"

# Problem 2

In [2]:
//Import
import org.apache.spark.graphx._

Intitializing Scala interpreter ...

Spark Web UI available at http://bd-hm:8088/proxy/application_1571094080322_0008
SparkContext available as 'sc' (version = 2.4.4, master = yarn, app id = application_1571094080322_0008)
SparkSession available as 'spark'


import org.apache.spark.graphx._


In [3]:
//Extract data
val rdd= sc.textFile("hdfs://bd-hm:9000/hadoop-user/data/higgs-social_network.edgelist")
rdd.take(5).foreach(println)

1 2
1 3
1 4
1 5
1 6


rdd: org.apache.spark.rdd.RDD[String] = hdfs://bd-hm:9000/hadoop-user/data/higgs-social_network.edgelist MapPartitionsRDD[1] at textFile at <console>:29


In [4]:
//Create pair rdd of form (userid_1, userid2)
val edges=rdd.map(line=>line.split(" ")).map(x=>(x(0).toLong,x(1).toLong))
edges.take(4).foreach(println)

(1,2)
(1,3)
(1,4)
(1,5)


edges: org.apache.spark.rdd.RDD[(Long, Long)] = MapPartitionsRDD[3] at map at <console>:30


In [5]:
//Create graph
val graph = Graph.fromEdgeTuples(edges,null)

graph: org.apache.spark.graphx.Graph[Null,Int] = org.apache.spark.graphx.impl.GraphImpl@4a33e36e


In [6]:
//Find number of vertices
val nv=graph.numVertices

nv: Long = 456626


In [7]:
//Extract in-degrees for each node
val inDeg=graph.inDegrees
inDeg.take(10).foreach(println)

(185012,15)
(129434,39)
(182316,15)
(194402,2)
(199516,8)
(332918,1)
(170792,3)
(307248,2)
(32676,53)
(38926,28)


inDeg: org.apache.spark.graphx.VertexRDD[Int] = VertexRDDImpl[18] at RDD at VertexRDD.scala:57


In [8]:
//Extract only the in-degrees
val onlyDeg=inDeg.map(x=>x.toString.split(",")(1).replaceAll("\\)", ""))
//Count # of occurences of each in-degree value
val countDeg=onlyDeg.map(x=>(x,1)).reduceByKey((v1,v2)=>v1+v2)
countDeg.take(25).foreach(println)

(273,37)
(1863,1)
(1148,1)
(1119,1)
(736,4)
(3638,1)
(1245,1)
(312,19)
(62,601)
(1184,1)
(1625,1)
(1566,1)
(734,6)
(3454,1)
(2329,1)
(279,32)
(1803,1)
(2033,1)
(479,5)
(2114,1)
(3939,1)
(2303,1)
(2277,1)
(989,3)
(642,9)


onlyDeg: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[19] at map at <console>:30
countDeg: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[21] at reduceByKey at <console>:32


In [9]:
//Output file of form (k, p(k)) where k is in-degree value and p(k) is probability
val pk = countDeg.map(x=>(x.toString.split(",")(0).replaceAll("\\(", ""), x.toString.split(",")(1).replaceAll("\\)", "").toFloat/456626))

pk.take(10).foreach(println)
//Save p(k) file
pk.saveAsTextFile("file:///home/administrator/Desktop/edgelist_output")

(273,8.102912E-5)
(1863,2.189976E-6)
(1148,2.189976E-6)
(1119,2.189976E-6)
(736,8.759904E-6)
(3638,2.189976E-6)
(1245,2.189976E-6)
(312,4.1609546E-5)
(62,0.0013161756)
(1184,2.189976E-6)


pk: org.apache.spark.rdd.RDD[(String, Float)] = MapPartitionsRDD[22] at map at <console>:30
